In [1]:
import pandas as pd
from recipe_scrapers import scrape_me
import time
import random
import json
import pymongo
from tqdm.notebook import tqdm

In [2]:
def wait():
    
    x = random.randrange(50, 200, 1)/100
    print(f"Waiting for {x} Seconds.", end='\r')
    time.sleep(x)

In [3]:
# Db Information
client = pymongo.MongoClient("mongodb://localhost:27017/")
db = client['food_analysis']
urls = db['urls']
recipes = db['recipes']

# Scraper
Take the url database and start retrieving the actual recipe contents to store in a new database dedicated to the recipes. Given that this database will form the basis for an actual app and analysis the key unique id will be generated by Mongo and keep close tabs with the url database to ensure no read recipes are reread.

In [26]:
eligible_urls = [x["_id"] for x in list(urls.find({"read":False}, {"_id":1}))]
eligible_urls = sorted(eligible_urls, key = lambda x: random.random())

for url in tqdm(eligible_urls):
    
    # Obtain existing data
    print(url)
    row = list(urls.find({"_id":url}))[0]
    
    
    try:
        if url[0] != 'h':
            url = "https:" + url

        # Pause and then scrape
        wait()
        scraper = scrape_me(url)

        # Attempt each piece of data

        # Title
        try:
            title = scraper.title()
        except:
            try:
                title = row['name']
            except:
                title = None

        # Total Time
        try:
            total_time = scraper.total_time()
        except:
            total_time = None

        # Yields
        try:
            yields = scraper.yields()
        except:
            yields = None

        # Ingredients
        try:
            ingredients = scraper.ingredients()
        except:
            ingredients = []

        # Instructions
        try:
            instructions = scraper.instructions().encode('utf-8', 'surrogatepass')
        except:
            instructions = None

        # Image
        try:
            image = scraper.image()
        except:
            image = None

        # Ratings
        try:
            rating = scraper.ratings()
        except:
            rating = None

        # Author
        try:
            author = scraper.author()
        except:
            try:
                author = row['author']
            except:
                author = None

        # Reviews
        try:
            reviews = scraper.reviews()
        except:
            reviews = None

        # Insert new data
        recipes.insert_one({"title":title,
                            "total_time":total_time,
                            "yields":yields,
                            "ingredients":ingredients,
                            "instructions":instructions,
                            "image":image,
                            "rating":rating,
                            "author":author,
                            "reviews":reviews,
                            "source":row['source'],
                            "url":row["_id"]})

        # Mark as read
        query = {"_id":url}
        newvalues = {"$set":{"read":True}}
        urls.update_one(query, newvalues)
        
    except:
        
        # Mark as Error
        print("Error encountered.")
        query = {"_id":url}
        newvalues = {"$set":{"error":True}}
        urls.update_one(query, newvalues)

https://www.food.com/recipe/caribbean-chicken-soup-with-bananas-174961
https://www.food.com/recipe/oaty-chocolate-chunk-cookies-330533
https://www.food.com/recipe/lunchbox-hummus-vegetable-sandwich-267593
https://www.food.com/recipe/low-fat-skinny-cow-chocolate-cake-sauce-412461
https://www.southernliving.com/recipes/fall-9x13-recipes
https://www.food.com/recipe/charoset-299024
https://www.food.com/recipe/gluten-dairy-cane-sugar-free-blueberry-muffins-431962
https://www.cookstr.com/recipes/fresh-linguine-with-heirloom-tomatoes-ricotta-salata-and-basil-pesto
https://www.epicurious.com/recipes/food/views/watermelon-berry-granita-103805
https://www.food.com/recipe/oven-fried-pounded-chicken-337452
https://www.food.com/recipe/pa-dutch-chicken-pot-pie-noodle-soup-185446
https://www.southernliving.com/recipes/cucumber-chile-paletas-recipe
https://www.closetcooking.com/blue-cheese-dip/
https://www.food.com/recipe/white-bean-and-roasted-eggplant-hummus-baba-ghanoush-427417
https://www.epicurio

https://www.closetcooking.com/pork-chops-with-mushrooms-dill-and-sour/
https://www.food.com/recipe/creamy-chicken-and-rice-bake-162115
https://www.food.com/recipe/cream-puffs-16207
https://www.food.com/recipe/extreme-chocolate-cake-172939
https://www.food.com/recipe/dried-beef-cheese-ball-416685
https://www.cookstr.com/recipes/bibimbabkorean-rice-with-eg
https://www.epicurious.com/recipes/food/views/hard-boiled-egg-dressing-with-tarragon-and-cornichons-11507
https://www.food.com/recipe/basic-biscotti-31230
https://www.epicurious.com/recipes/food/views/stone-fruit-clafoutis
https://www.food.com/recipe/guacamole-135892
https://www.food.com/recipe/easy-lemon-bars-150777
https://www.food.com/recipe/lebanese-spiced-potatoes-batata-harra-443414
https://www.epicurious.com/recipes/food/views/pecan-crusted-salmon-with-sorrel-sauce-2272
https://www.food.com/recipe/creamy-italian-salad-dressing-245393
https://www.epicurious.com/recipes/food/views/rosemary-mojito-winter-drinks
https://www.food.com

https://www.food.com/recipe/chili-pineapple-61861
https://www.food.com/recipe/colombian-pound-cake-173432
https://www.epicurious.com/recipes/food/views/the-green-lantern-242851
https://www.epicurious.com/recipes/food/views/cardamom-pistachio-carrot-cake
https://www.food.com/recipe/curried-sweet-potatoes-cauliflower-and-green-beans-231629
//www.foodnetwork.com/recipes/bobby-deen/bobbys-baked-tilapia-recipe-1945406
https://www.foodrepublic.com/recipes/a-cafe-de-paris-butter-recipe/
https://www.food.com/recipe/scottish-lorne-sausages-square-breakfast-sausage-389024
https://www.food.com/recipe/chocolate-blackout-cake-245144
https://www.epicurious.com/recipes/food/views/mashed-jerusalem-artichokes-101053
https://www.epicurious.com/recipes/food/views/prosciutto-pear-and-fennel-salad-1032
https://www.food.com/recipe/apple-bundt-cake-510256
https://www.food.com/recipe/fruit-carrot-smoothie-150556
https://www.food.com/recipe/powdered-laundry-soap-107987
https://www.food.com/recipe/avocado-and-a

https://www.food.com/recipe/the-coach-houses-black-bean-soup-460269
https://www.epicurious.com/recipes/food/views/soy-braised-pork-country-ribs-with-carrots-and-turnips-361251
https://www.food.com/recipe/swedish-apple-pie-143314
https://www.food.com/recipe/quick-chocolate-crinkles-43200
//www.foodnetwork.com/recipes/tyler-florence/taro-infused-with-lemon-grass-and-ginger-recipe2-1947541
https://www.food.com/recipe/filipino-itlog-na-maalat-at-kamatis-egg-and-tomato-salad-532382
https://www.epicurious.com/recipes/food/views/chicken-with-port-mushroom-sauce-773
https://www.food.com/recipe/duck-with-ginger-and-lime-104404
https://www.food.com/recipe/rice-cheese-balls-326593
https://www.food.com/recipe/best-ever-cranberry-salad-255316
https://www.food.com/recipe/katsu-don-pork-cutlet-donburi-24280
https://www.epicurious.com/recipes/food/views/feta-corn-muffins-13097
https://www.epicurious.com/recipes/food/views/french-potato-salad-101986
https://www.food.com/recipe/juicy-fruit-salad-219136


https://www.epicurious.com/recipes/food/views/lentil-and-sweet-red-pepper-soup-with-cumin-and-black-pepper-106252
https://www.epicurious.com/recipes/food/views/-em-polow-em-persian-rice-with-pistachios-and-dill-352469
https://www.food.com/recipe/silver-lining-salad-210427
https://www.food.com/recipe/any-kind-muffins-gluten-free-360337
https://www.food.com/recipe/orange-cream-cheese-frosting-466179
//www.foodnetwork.com/recipes/giada-de-laurentiis/sweet-fresh-fettuccini-recipe-1946731
https://www.food.com/recipe/linzer-torte-bars-77493
https://www.food.com/recipe/southern-cornbread-sage-dressing-529220
https://www.epicurious.com/recipes/food/views/stuffed-leg-of-lamb-fakdeh-mehshi-khodra-233061
https://www.food.com/recipe/cream-corn-casserole-165699
https://www.food.com/recipe/chicken-and-rice-casserole-with-chilies-corn-and-black-beans-231241
https://www.epicurious.com/recipes/food/views/sesame-pea-shoot-salad-12059
https://www.food.com/recipe/roasted-asparagus-pasta-with-garlic-butter

...

https://www.southernliving.com/syndication/ambrosia-cocktail-0
https://www.epicurious.com/recipes/food/views/jamaican-jerk-chicken-234807
//www.foodnetwork.com/recipes/giada-de-laurentiis/open-faced-finger-sandwiches-3620262
https://www.epicurious.com/recipes/food/views/zucchini-basil-soup-242831
https://www.food.com/recipe/espresso-cinnamon-muffins-468955
https://www.food.com/recipe/artichoke-leek-and-ham-quiche-filling-200905
https://www.food.com/recipe/orange-berry-swirl-418727
https://www.epicurious.com/recipes/food/views/peanut-dipping-sauce-10432
https://www.food.com/recipe/malay-pineapple-curry-pajri-nenas-154301
https://www.food.com/recipe/easy-bake-oven-english-muffin-pizza-151887
https://www.epicurious.com/recipes/food/views/roast-chicken-with-mushroom-stuffing-and-sauce-104080
https://www.food.com/recipe/bruschetta-with-pesto-tomatoes-and-thingies-225571
https://www.food.com/recipe/caraway-cabbage-175310
https://www.food.com/recipe/warm-lentil-ham-salad-with-dijon-cream-1250

https://www.food.com/recipe/flourless-chocolate-snowball-cookies-469463
https://www.food.com/recipe/vegetable-and-pasta-soup-281937
https://www.food.com/recipe/hawaiian-chicken-salad-456627
https://www.food.com/recipe/diet-carrot-souffle-112279
https://www.food.com/recipe/cranberry-streusel-muffins-138774
https://www.food.com/recipe/fiesta-cheese-rice-282714
https://www.food.com/recipe/moms-hot-skillet-bacon-cole-slaw-with-apples-99749
https://www.food.com/recipe/dutch-uitsmijter-fried-ham-and-eggs-with-mustard-cheese-158213
https://www.gimmesomeoven.com/quinoa-tortilla-soup/
https://www.food.com/recipe/broccoli-cauliflower-salad-267259
https://www.food.com/recipe/manhattan-seafood-chowder-107404
https://www.food.com/recipe/pork-salad-sandwiches-with-maple-dijon-dressing-121161
https://www.epicurious.com/recipes/food/views/avocado-and-tangerine-salad-with-jalapeno-vinaigrette-51175640
https://www.epicurious.com/recipes/food/views/veal-chops-with-asparagus-and-morels-242147
https://www.

https://www.food.com/recipe/hamburger-soup-63657
https://www.food.com/recipe/easy-berry-freezer-jam-224265
https://www.food.com/recipe/quick-n-easy-strawberry-and-banana-smoothie-122568
https://www.food.com/recipe/broccoli-rice-hot-dish-385378
https://www.food.com/recipe/wild-duck-with-pecan-stuffing-74889
https://www.epicurious.com/recipes/food/views/almond-crusted-shrimp-cakes-with-lemon-soy-mayonnaise-232783
https://www.epicurious.com/recipes/food/views/roast-veal-brisket-with-marsala-mushroom-sauce-2893
https://www.epicurious.com/recipes/food/views/minted-lobster-salad-in-pitas-12392
https://www.food.com/recipe/hungry-girl-buffalo-chicken-dip-413265
https://www.food.com/recipe/chai-spiced-double-apple-pie-402613
https://www.food.com/recipe/mexican-tres-leche-cake-26014
https://www.food.com/recipe/asian-shrimp-soup-398165
https://www.food.com/recipe/teriyaki-marinade-97994
https://www.food.com/recipe/throw-away-the-bread-machine-instructions-white-bread-339905
https://www.food.com/r

https://www.food.com/recipe/finadene-sauce-38533
https://www.food.com/recipe/black-beans-and-peaches-135561
https://www.food.com/recipe/donut-bread-pudding-with-butter-rum-sauce-213000
https://www.food.com/recipe/texas-brisket-w-coffee-and-beer-mop-sauce-218289
https://www.food.com/recipe/cheese-steak-soup-279635
https://www.food.com/recipe/mietta-family-eggplant-for-antipasto-288127
//www.foodnetwork.com/recipes/guy-fieri/pastrami-burger-3362868
https://www.closetcooking.com/baby-bok-choy-with-shiitake-mushrooms/
https://www.food.com/recipe/corned-beef-mini-loaves-498061
https://www.epicurious.com/recipes/food/views/mexican-pasta-surprise
https://www.food.com/recipe/sugared-grapes-39947
https://www.food.com/recipe/chicken-and-soba-noodle-salad-74679
https://www.epicurious.com/recipes/food/views/spice-rubbed-chicken-breasts-with-lemon-shallot-sauce-106642
https://www.food.com/recipe/slow-cooker-beef-stew-518027
https://www.food.com/recipe/aloha-andrea-354084
https://www.food.com/recipe

KeyboardInterrupt: 

# Database Cleaning and Maintenance

In [24]:
# Remove Duplica
list(recipes.find({}))

[{'_id': ObjectId('5f20f73c3aa0b95774cf60a7'),
  'title': 'Quick Pizza Sauce With Fennel',
  'total_time': 5,
  'yields': '8 serving(s)',
  'ingredients': ['1 (8 ounce) can tomato sauce',
   '1 (3 ounce) can mushroom pieces, undrained',
   '1 teaspoon garlic powder (to taste)',
   '1⁄2 teaspoon fennel seed (to taste)',
   '1⁄2 teaspoon dried oregano, to taste',
   '1 pinch black pepper (to taste)'],
  'instructions': 'Drain mushrooms, reserving liquid.  Set mushrooms aside for another use (topping, perhaps).\r\nCombine reserved mushroom liquid with all other ingredients.\r\nUse on pizza of your choice.',
  'image': 'https://img.sndimg.com/food/image/upload/q_92,fl_progressive,w_1200,c_scale/v1/gk-static/fdc-new/img/fdc-shareGraphic.png',
  'rating': 5.0,
  'author': None,
  'reviews': None,
  'source': 'food_com',
  'url': 'https://www.food.com/recipe/quick-pizza-sauce-with-fennel-409930'},
 {'_id': ObjectId('5f20f73d3aa0b95774cf60a8'),
  'title': "Elephant's Mudbath",
  'total_time': 